In [5]:
import pandas as pd
import numpy as np
import os
from source._helpersV4 import logSave,load_parameters

# Getting best classifier based on the centremost sensor (evolution set)

In [31]:
#Get the best model for the centremost sensor
centremost = 27
arch_type = 'pre_adjw'

p_path = 'experiments/shms/shms_p.json'
p = load_parameters(p_path)

In [32]:
# l_diff_type = ['diff','non-diff']
l_exp = [1,2,3]
l_inputs = [10,20,30,50]#,100]
l_gen = [str(i).zfill(4) for i in range(10,301,10)]
l_filter = ['none','mio','softmax','none','mio','softmax','none','mio','softmax','mio','softmax']
l_loss_f = ['mul_f1','mul_f1','mul_f1','mul_ap','mul_ap','mul_ap','mae','mae','mae','mul_ce','mul_ce']
l_c = []
for exp in l_exp:
    for inputs in l_inputs:
        for filter,loss_f in zip(l_filter,l_loss_f):
            if any(lf in loss_f for lf in ['ap','f1']):
                diff_type = 'non-diff'
                opt_direction = 'maximize'
            elif any(lf in loss_f for lf in ['mae','ce']):
                diff_type = 'diff'
                opt_direction = 'minimize'

            config = {}
            config['fname'] = '{}shms_s{}_{}/results/evolset_ch{}/{}/exp{}_ins{}_{}_{}.csv'.\
            format(p['experiment_path'],centremost,diff_type,centremost,arch_type,exp,inputs,filter,loss_f)

            l_c += [config]

### defining functions

In [33]:
def get_evolutionset_scores(c_l):
    check_if_exists(c_l)
    df_scores = pd.DataFrame({})
    for c in c_l:        
        tmp = pd.read_csv(c['fname'])
        df_scores = pd.concat([df_scores,tmp])  
    df_scores.reset_index(inplace=True,drop=True)
    return df_scores

def check_if_exists(c_l):
    files_missing = []
    for c in c_l:
        if not os.path.exists(c['fname']):
            files_missing += [c['fname']]
    if files_missing == []:
        print('checked all architecture files exists for {}'.format(p['data_path']))
    else:
        print('List of filess missing to run cma adjustment:')
        for item in files_missing:
            print(item)
        raise ValueError('Correct missing files before running cma adjustment script')
    

In [34]:
df_scores = get_evolutionset_scores(l_c)
filter = df_scores.gen == 300
df_300 = df_scores[filter]
# df_300 = df_scores
df_scores

checked all architecture files exists for dataset/shms_data/


,centremost,sensor,exp,inputs,filter_lossf,gen,AP_macro,precision_macro,recall_macro,f1-score_macro,load_time,pred_time,scoring_time
0,27,27,1,10,none_mul_f1,10,0.353377,0.664436,0.606527,0.634162,3.614251e-02,0.025386,0.000975
1,27,27,1,10,none_mul_f1,20,0.374627,0.886062,0.699039,0.781518,3.614251e-02,0.028854,0.000967
2,27,27,1,10,none_mul_f1,30,0.536346,0.982408,0.951641,0.966780,3.614251e-02,0.032058,0.001159
3,27,27,1,10,none_mul_f1,40,0.536346,0.982408,0.951641,0.966780,3.614251e-02,0.032058,0.001159
4,27,27,1,10,none_mul_f1,50,0.521263,0.994977,0.965778,0.980160,3.614251e-02,0.036339,0.000992
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,27,27,3,50,softmax_mul_ce,260,0.395911,0.445493,0.499996,0.471174,4.169686e-07,0.018810,0.001016
3956,27,27,3,50,softmax_mul_ce,270,0.395911,0.445493,0.499996,0.471174,4.169686e-07,0.018810,0.001016
3957,27,27,3,50,softmax_mul_ce,280,0.395911,0.445493,0.499996,0.471174,4.169686e-07,0.018810,0.001016
3958,27,27,3,50,softmax_mul_ce,290,0.395911,0.445493,0.499996,0.471174,4.169686e-07,0.018810,0.001016


### Getting the top 10 models with evolution set

In [35]:
aux = df_300.groupby(['inputs','filter_lossf','gen']).mean()
top_10 = aux.sort_values('f1-score_macro',ascending=False).iloc[:10,:]
top_10

centremost  sensor  exp  AP_macro  precision_macro  \
inputs filter_lossf   gen                                                       
10     mio_mul_ap     300        27.0    27.0  2.0  0.777683         0.993115   
20     none_mul_f1    300        27.0    27.0  2.0  0.818540         0.994250   
       mio_mul_f1     300        27.0    27.0  2.0  0.818540         0.994250   
       softmax_mul_f1 300        27.0    27.0  2.0  0.673850         0.988499   
10     mio_mul_f1     300        27.0    27.0  2.0  0.694340         0.985950   
       none_mul_f1    300        27.0    27.0  2.0  0.694340         0.985950   
30     softmax_mul_f1 300        27.0    27.0  2.0  0.797837         0.986892   
10     softmax_mul_f1 300        27.0    27.0  2.0  0.571825         0.977462   
50     softmax_mul_f1 300        27.0    27.0  2.0  0.842477         0.971010   
20     mio_mul_ap     300        27.0    27.0  2.0  0.780932         0.964894   

                           recall_macro  f1-score_macro     load_time  \
inputs filter_lossf   gen                                               
10     mio_mul_ap     300      0.974721        0.983828  4.122836e-07   
20     none_mul_f1    300      0.967639        0.980761  4.710739e-03   
       mio_mul_f1     300      0.967639        0.980761  4.149016e-07   
       softmax_mul_f1 300      0.952238        0.969940  4.146869e-07   
10     mio_mul_f1     300      0.950046        0.967411  4.379932e-07   
       none_mul_f1    300      0.950046        0.967411  1.204785e-02   
30     softmax_mul_f1 300      0.917031        0.950667  4.174628e-07   
10     softmax_mul_f1 300      0.912423        0.943541  4.165651e-07   
50     softmax_mul_f1 300      0.915414        0.942339  4.237518e-07   
20     mio_mul_ap     300      0.914480        0.938876  4.710578e-07   

                           pred_time  scoring_time  
inputs filter_lossf   gen                           
10     mio_mul_ap     300   0.028891      0.001015  
20     none_mul_f1    300   0.042140      0.001001  
       mio_mul_f1     300   0.041380      0.000914  
       softmax_mul_f1 300   0.047330      0.001050  
10     mio_mul_f1     300   0.031567      0.001012  
       none_mul_f1    300   0.039603      0.000949  
30     softmax_mul_f1 300   0.033672      0.000931  
10     softmax_mul_f1 300   0.023097      0.001231  
50     softmax_mul_f1 300   0.038436      0.000939  
20     mio_mul_ap     300   0.029427      0.000987

### Getting the best model for each loss function using the evolution set

In [36]:
loss_f_l = ['mae','mul_ce','mul_f1','mul_ap']

top_by_lossf = pd.DataFrame({})

for lossf in loss_f_l:
    mask = df_300['filter_lossf'].str.contains(lossf, na=False)
    aux = df_300[mask].groupby(['inputs','filter_lossf','gen']).mean()
    top = aux.sort_values('f1-score_macro',ascending=False)
    inputs,flf,gen = top.index[0]
    mask = (df_300['inputs'] == inputs) & (df_300['filter_lossf'] == flf) & (df_300['gen'] == gen)
    top_by_lossf = pd.concat([top_by_lossf,df_300[mask]],axis=0)

top_by_lossf.reset_index(inplace=True,drop=True)
top_by_lossf.groupby(['inputs','filter_lossf','gen']).mean()


centremost  sensor  exp  AP_macro  precision_macro  \
inputs filter_lossf gen                                                       
10     mio_mae      300        27.0    27.0  2.0  0.670230         0.888431   
       mio_mul_ap   300        27.0    27.0  2.0  0.777683         0.993115   
20     mio_mul_ce   300        27.0    27.0  2.0  0.542503         0.903867   
       mio_mul_f1   300        27.0    27.0  2.0  0.818540         0.994250   

                         recall_macro  f1-score_macro     load_time  \
inputs filter_lossf gen                                               
10     mio_mae      300      0.839064        0.862652  5.949625e-07   
       mio_mul_ap   300      0.974721        0.983828  4.122836e-07   
20     mio_mul_ce   300      0.792663        0.842453  4.307056e-07   
       mio_mul_f1   300      0.967639        0.980761  4.149016e-07   

                         pred_time  scoring_time  
inputs filter_lossf gen                           
10     mio_mae      300   0.034801      0.000904  
       mio_mul_ap   300   0.028891      0.001015  
20     mio_mul_ce   300   0.039976      0.001016  
       mio_mul_f1   300   0.041380      0.000914

In [26]:
# loss_f_l = ['mae','mul_ce','mul_f1','mul_ap']

# top_by_lossf = pd.DataFrame({})

# for lossf in loss_f_l:
#     mask = df_scores['filter_lossf'].str.contains(lossf, na=False)
#     aux = df_scores[mask].groupby(['inputs','filter_lossf','gen']).mean()
#     top = aux.sort_values('macro f1-score',ascending=False)
#     inputs,flf,gen = top.index[0]
#     mask = (df_scores['inputs'] == inputs) & (df_scores['filter_lossf'] == flf) & (df_scores['gen'] == gen)
#     top_by_lossf = pd.concat([top_by_lossf,df_scores[mask]],axis=0)

# top_by_lossf.reset_index(inplace=True,drop=True)
# top_by_lossf.groupby(['inputs','filter_lossf','gen']).mean()


#### saving dataframe with parameters for best by loss function 

In [10]:
fname = '{}evolset_s{}_bestby_lossf.csv'.format(p['experiment_path'],centremost)
idx = top_by_lossf.groupby(['inputs','filter_lossf','gen']).mean().index
col = ['centremost','inputs','filter','lossf','gen']
df_to_save = pd.DataFrame({},columns=col)
for inputs,flf,gen in idx:
    filt,lossf = flf.split('_', maxsplit=1)
    df_to_save = pd.concat([df_to_save,pd.DataFrame([[centremost,inputs,filt,lossf,gen]],columns=col)],axis=0)

df_to_save.reset_index(inplace=True,drop=True)
df_to_save.to_csv(fname,index=False)

In [11]:
centremost = 27
df_fname = f'evolset_s{centremost}_bestby_lossf.csv'

fname = p['experiment_path']+df_fname

def lists_from_bestby_df(fname):
    df = pd.read_csv(fname)
    return df['inputs'].to_list(),df['filter'].to_list(),df['lossf'].to_list(),df['gen'].to_list()

lists_from_bestby_df(fname)

([10, 10, 20, 20],
 ['mio', 'mio', 'mio', 'mio'],
 ['mae', 'mul_ap', 'mul_ce', 'mul_f1'],
 [300, 300, 300, 300])

## Getting results for each cluster

In [6]:
def lists_from_bestby(p,centremost):
    fname = '{}evolset_s{}_bestby_lossf.csv'.format(p['experiment_path'],centremost)
    df = pd.read_csv(fname)
    return df['inputs'].to_list(),df['filter'].to_list(),df['lossf'].to_list(),df['gen'].to_list()

In [7]:
def get_evolutionset_scores(c_l):
    check_if_exists(c_l)
    df_scores = pd.DataFrame({})
    for c in c_l:        
        tmp = pd.read_csv(c['fname'])
        df_scores = pd.concat([df_scores,tmp])  
    df_scores.reset_index(inplace=True,drop=True)
    return df_scores

def check_if_exists(c_l):
    files_missing = []
    for c in c_l:
        if not os.path.exists(c['fname']):
            files_missing += [c['fname']]
    if files_missing == []:
        print('checked all architecture files exists for {}'.format(p['data_path']))
    else:
        print('List of filess missing to run cma adjustment:')
        for item in files_missing:
            print(item)
        raise ValueError('Correct missing files before running cma adjustment script')
    

In [8]:
def get_scores(p,centremost,post_cma):
    l_exp = [1,2,3]
    # l_inputs = [10,20,30,50,100]
    l_gen = [str(i).zfill(4) for i in range(10,301,10)]
    # l_filter = ['none','mio','softmax','none','mio','softmax','none','mio','softmax','mio','softmax']
    # l_loss_f = ['mul_f1','mul_f1','mul_f1','mul_ap','mul_ap','mul_ap','mae','mae','mae','mul_ce','mul_ce']
    l_inputs,l_filter,l_loss_f,_ = lists_from_bestby(p,centremost)
    print(l_inputs,l_filter,l_loss_f)
    if centremost == 4:
        l_sensor = [1,2,3,4,29,30]#[30]
    elif centremost == 14:
        l_sensor = [13,14,15,16,17,18,19]
    elif centremost == 27:
        l_sensor = [27]#[25,26,27,28]
    elif centremost == 35:
        l_sensor = [33,34,35]
    l_c = []
    for sensor in l_sensor:
        for exp in l_exp:
            for inputs,filter,loss_f in zip(l_inputs,l_filter,l_loss_f):
                if any(lf in loss_f for lf in ['ap','f1']):
                    diff_type = 'non-diff'
                    opt_direction = 'maximize'
                elif any(lf in loss_f for lf in ['mae','ce']):
                    diff_type = 'diff'
                    opt_direction = 'minimize'
    
                config = {}
                config['fname'] = '{}shms_s{}_{}/results/completeset_s{}/{}/{}/exp{}_ins{}_{}_{}.csv'.\
                format(p['experiment_path'],centremost,diff_type,sensor,arch_type,post_cma,exp,inputs,filter,loss_f)
    
                l_c += [config]

    return get_evolutionset_scores(l_c)



### Cluster 1 centremost 4

In [5]:
#Get the best model for the centremost sensor
centremost = 4
arch_type = 'pre_adjw'

p_path = 'experiments/shms/shms_p.json'
p = load_parameters(p_path)

In [6]:
df_scores_c1 = get_scores(p,centremost)
df_scores_c1.head(5)

checked all architecture files exists for dataset/shms_data/


,centremost,sensor,exp,inputs,filter_lossf,gen,AP_macro,precision_macro,recall_macro,f1-score_macro,load_time,pred_time,scoring_time
0,4,1,1,10,mio_mae,300,0.878329,0.971595,0.897020,0.932819,2.337511,0.084920,0.132084
1,4,1,1,10,mio_mul_ap,300,0.901604,0.990178,0.902386,0.944246,1.832394,0.141387,0.134601
2,4,1,1,10,mio_mul_ce,300,0.870615,0.962011,0.898923,0.929398,1.857374,0.050136,0.131994
3,4,1,1,10,mio_mul_f1,300,0.709006,0.798867,0.892808,0.843229,1.781471,0.111014,0.132794
4,4,1,1,10,mio_mae,300,0.878329,0.971595,0.897020,0.932819,2.337511,0.084920,0.132084


#### View of scores for each sensor in cluster 1

In [7]:
df_scores_c1.groupby(['sensor','inputs','filter_lossf','gen']).mean()

centremost  exp  AP_macro  precision_macro  \
sensor inputs filter_lossf gen                                               
1      10     mio_mae      300         4.0  2.0  0.827509         0.920668   
              mio_mul_ap   300         4.0  2.0  0.898443         0.989974   
              mio_mul_ce   300         4.0  2.0  0.884935         0.980025   
              mio_mul_f1   300         4.0  2.0  0.775767         0.868959   
2      10     mio_mae      300         4.0  2.0  0.827509         0.920668   
              mio_mul_ap   300         4.0  2.0  0.898443         0.989974   
              mio_mul_ce   300         4.0  2.0  0.884935         0.980025   
              mio_mul_f1   300         4.0  2.0  0.775767         0.868959   
3      10     mio_mae      300         4.0  2.0  0.827509         0.920668   
              mio_mul_ap   300         4.0  2.0  0.898443         0.989974   
              mio_mul_ce   300         4.0  2.0  0.884935         0.980025   
              mio_mul_f1   300         4.0  2.0  0.775767         0.868959   
4      10     mio_mae      300         4.0  2.0  0.827509         0.920668   
              mio_mul_ap   300         4.0  2.0  0.898443         0.989974   
              mio_mul_ce   300         4.0  2.0  0.884935         0.980025   
              mio_mul_f1   300         4.0  2.0  0.775767         0.868959   
29     10     mio_mae      300         4.0  2.0  0.827509         0.920668   
              mio_mul_ap   300         4.0  2.0  0.898443         0.989974   
              mio_mul_ce   300         4.0  2.0  0.884935         0.980025   
              mio_mul_f1   300         4.0  2.0  0.775767         0.868959   
30     10     mio_mae      300         4.0  2.0  0.827509         0.920668   
              mio_mul_ap   300         4.0  2.0  0.898443         0.989974   
              mio_mul_ce   300         4.0  2.0  0.884935         0.980025   
              mio_mul_f1   300         4.0  2.0  0.775767         0.868959   

                                recall_macro  f1-score_macro  load_time  \
sensor inputs filter_lossf gen                                            
1      10     mio_mae      300      0.897547        0.906746   0.779171   
              mio_mul_ap   300      0.899401        0.942516   0.610798   
              mio_mul_ce   300      0.895339        0.935709   0.619125   
              mio_mul_f1   300      0.895006        0.879631   0.593824   
2      10     mio_mae      300      0.897547        0.906746   0.738410   
              mio_mul_ap   300      0.899401        0.942516   0.609276   
              mio_mul_ce   300      0.895339        0.935709   0.593853   
              mio_mul_f1   300      0.895006        0.879631   0.623041   
3      10     mio_mae      300      0.897547        0.906746   0.736770   
              mio_mul_ap   300      0.899401        0.942516   0.592694   
              mio_mul_ce   300      0.895339        0.935709   0.608204   
              mio_mul_f1   300      0.895006        0.879631   0.623718   
4      10     mio_mae      300      0.897547        0.906746   0.723536   
              mio_mul_ap   300      0.899401        0.942516   0.609162   
              mio_mul_ce   300      0.895339        0.935709   0.609393   
              mio_mul_f1   300      0.895006        0.879631   0.605100   
29     10     mio_mae      300      0.897547        0.906746   0.713871   
              mio_mul_ap   300      0.899401        0.942516   0.595999   
              mio_mul_ce   300      0.895339        0.935709   0.594227   
              mio_mul_f1   300      0.895006        0.879631   0.610398   
30     10     mio_mae      300      0.897547        0.906746   0.733268   
              mio_mul_ap   300      0.899401        0.942516   0.604099   
              mio_mul_ce   300      0.895339        0.935709   0.622302   
              mio_mul_f1   300      0.895006        0.879631   0.626592   

                                pred_time  scoring_t

### Cluster 2 centremost 14

In [25]:
#Get the best model for the centremost sensor
centremost = 14
arch_type = 'pre_adjw'

p_path = 'experiments/shms/shms_p.json'
p = load_parameters(p_path)

In [26]:
df_scores_c2 = get_scores(p,centremost)
df_scores_c2.head(5)

[10, 30, 30, 50] ['softmax', 'softmax', 'softmax', 'mio'] ['mul_ce', 'mae', 'mul_f1', 'mul_ap']
checked all architecture files exists for dataset/shms_data/


,centremost,sensor,exp,inputs,filter_lossf,gen,AP_macro,precision_macro,recall_macro,f1-score_macro,load_time,pred_time,scoring_time
0,14,13,1,10,softmax_mul_ce,300,0.999996,0.817931,0.837809,0.827750,2.340890e+00,0.146792,0.119575
1,14,13,1,30,softmax_mae,300,0.806974,0.817761,0.837474,0.827500,2.103706e+00,0.164988,0.119624
2,14,13,1,30,softmax_mul_f1,300,0.964679,0.856016,0.897525,0.876280,2.118288e+00,0.166436,0.140767
3,14,13,1,50,mio_mul_ap,300,0.767301,0.816667,0.835307,0.825882,2.492867e+00,0.152086,0.101846
4,14,13,2,10,softmax_mul_ce,300,0.733489,0.795021,0.784806,0.789880,4.268174e-07,0.058116,0.102364


#### View of scores for each sensor in cluster 1

In [27]:
df_scores_c2.groupby(['sensor','inputs','filter_lossf','gen']).mean()

centremost  exp  AP_macro  precision_macro  \
sensor inputs filter_lossf   gen                                               
13     10     softmax_mul_ce 300        14.0  2.0  0.877634         0.806483   
       30     softmax_mae    300        14.0  2.0  0.639684         0.709186   
              softmax_mul_f1 300        14.0  2.0  0.899417         0.824711   
       50     mio_mul_ap     300        14.0  2.0  0.777253         0.823138   
14     10     softmax_mul_ce 300        14.0  2.0  0.877634         0.806483   
       30     softmax_mae    300        14.0  2.0  0.639684         0.709186   
              softmax_mul_f1 300        14.0  2.0  0.899417         0.824711   
       50     mio_mul_ap     300        14.0  2.0  0.777253         0.823138   
15     10     softmax_mul_ce 300        14.0  2.0  0.877634         0.806483   
       30     softmax_mae    300        14.0  2.0  0.639684         0.709186   
              softmax_mul_f1 300        14.0  2.0  0.899417         0.824711   
       50     mio_mul_ap     300        14.0  2.0  0.777253         0.823138   
16     10     softmax_mul_ce 300        14.0  2.0  0.877634         0.806483   
       30     softmax_mae    300        14.0  2.0  0.639684         0.709186   
              softmax_mul_f1 300        14.0  2.0  0.899417         0.824711   
       50     mio_mul_ap     300        14.0  2.0  0.777253         0.823138   
17     10     softmax_mul_ce 300        14.0  2.0  0.877634         0.806483   
       30     softmax_mae    300        14.0  2.0  0.639684         0.709186   
              softmax_mul_f1 300        14.0  2.0  0.899417         0.824711   
       50     mio_mul_ap     300        14.0  2.0  0.777253         0.823138   
18     10     softmax_mul_ce 300        14.0  2.0  0.877634         0.806483   
       30     softmax_mae    300        14.0  2.0  0.639684         0.709186   
              softmax_mul_f1 300        14.0  2.0  0.899417         0.824711   
       50     mio_mul_ap     300        14.0  2.0  0.777253         0.823138   
19     10     softmax_mul_ce 300        14.0  2.0  0.877634         0.806483   
       30     softmax_mae    300        14.0  2.0  0.639684         0.709186   
              softmax_mul_f1 300        14.0  2.0  0.899417         0.824711   
       50     mio_mul_ap     300        14.0  2.0  0.777253         0.823138   

                                  recall_macro  f1-score_macro  load_time  \
sensor inputs filter_lossf   gen                                            
13     10     softmax_mul_ce 300      0.812049        0.809210   0.780297   
       30     softmax_mae    300      0.563846        0.618639   0.701236   
              softmax_mul_f1 300      0.843085        0.833658   0.706096   
       50     mio_mul_ap     300      0.846924        0.834851   0.830956   
14     10     softmax_mul_ce 300      0.812049        0.809210   0.763008   
       30     softmax_mae    300      0.563846        0.618639   0.699355   
              softmax_mul_f1 300      0.843085        0.833658   0.727958   
       50     mio_mul_ap     300      0.846924        0.834851   0.803028   
15     10     softmax_mul_ce 300      0.812049        0.809210   0.753412   
       30     softmax_mae    300      0.563846        0.618639   0.704007   
              softmax_mul_f1 300      0.843085        0.833658   0.710342   
       50     mio_mul_ap     300      0.846924        0.834851   0.813885   
16     10     softmax_mul_ce 300      0.812049        0.809210   0.769976   
       30     softmax_mae    300      0.563846        0.618639   0.707385   
              softmax_mul_f1 300      0.843085        0.833658   0.718362   
       50     mio_mul_ap     300      0.846924        0.834851   0.822769   
17     10     softmax_mul_ce 300      0.812049        0.809210   0.751692   
       30     softmax_mae    300      0.563846        0.618639   0.712880   
              softmax_mul_f1 300      0.843085        0.833658   0.726430   
       50     mi

### Cluster 3 centremost 27

In [9]:
#Get the best model for the centremost sensor
centremost = 27
arch_type = 'pre_adjw'

p_path = 'experiments/shms/shms_p.json'
p = load_parameters(p_path)

In [11]:
df_scores_c3 = get_scores(p,centremost,'post_cma')
df_scores_c3.head(5)

[10, 10, 20, 20] ['mio', 'mio', 'mio', 'mio'] ['mae', 'mul_ap', 'mul_ce', 'mul_f1']
checked all architecture files exists for dataset/shms_data/


,centremost,sensor,exp,inputs,filter_lossf,gen,AP_macro,precision_macro,recall_macro,f1-score_macro,load_time,pred_time,scoring_time
0,27,27,1,10,mio_mae,300,0.404821,0.418967,0.500001,0.455911,2.534414e+00,0.066546,0.132778
1,27,27,1,10,mio_mul_ap,300,0.619479,0.667529,0.747583,0.705292,2.012542e+00,0.125393,0.133446
2,27,27,1,20,mio_mul_ce,300,0.657177,0.635532,0.749898,0.687995,2.177083e+00,0.114159,0.132435
3,27,27,1,20,mio_mul_f1,300,0.657155,0.723439,0.749994,0.736477,2.181153e+00,0.192875,0.134445
4,27,27,2,10,mio_mae,300,0.654822,0.683848,0.749860,0.715334,4.626966e-07,0.216228,0.135089


In [29]:
df_scores_c3 = get_scores(p,centremost)
df_scores_c3.head(5)

[10, 10, 20, 20] ['mio', 'mio', 'mio', 'mio'] ['mae', 'mul_ap', 'mul_ce', 'mul_f1']
checked all architecture files exists for dataset/shms_data/


,centremost,sensor,exp,inputs,filter_lossf,gen,AP_macro,precision_macro,recall_macro,f1-score_macro,load_time,pred_time,scoring_time
0,27,25,1,10,mio_mae,300,0.404921,0.414879,0.500000,0.453480,2.432729e+00,0.140607,0.139656
1,27,25,1,10,mio_mul_ap,300,0.616160,0.633859,0.747309,0.685925,1.917902e+00,0.174007,0.144430
2,27,25,1,20,mio_mul_ce,300,0.657275,0.635648,0.749887,0.688058,2.028924e+00,0.124773,0.145402
3,27,25,1,20,mio_mul_f1,300,0.648559,0.651803,0.749376,0.697192,2.093414e+00,0.232313,0.139807
4,27,25,2,10,mio_mae,300,0.657215,0.696178,0.749846,0.722016,4.055987e-07,0.321012,0.140575


#### View of scores for each sensor in cluster 3

In [30]:
df_scores_c3.groupby(['sensor','inputs','filter_lossf','gen']).mean()

centremost  exp  AP_macro  precision_macro  \
sensor inputs filter_lossf gen                                               
25     10     mio_mae      300        27.0  2.0  0.573145         0.646749   
              mio_mul_ap   300        27.0  2.0  0.647208         0.659467   
       20     mio_mul_ce   300        27.0  2.0  0.657198         0.666021   
              mio_mul_f1   300        27.0  2.0  0.654267         0.663500   
26     10     mio_mae      300        27.0  2.0  0.573145         0.646749   
              mio_mul_ap   300        27.0  2.0  0.647208         0.659467   
       20     mio_mul_ce   300        27.0  2.0  0.657198         0.666021   
              mio_mul_f1   300        27.0  2.0  0.654267         0.663500   
27     10     mio_mae      300        27.0  2.0  0.573145         0.646749   
              mio_mul_ap   300        27.0  2.0  0.647208         0.659467   
       20     mio_mul_ce   300        27.0  2.0  0.657198         0.666021   
              mio_mul_f1   300        27.0  2.0  0.654267         0.663500   
28     10     mio_mae      300        27.0  2.0  0.573145         0.646749   
              mio_mul_ap   300        27.0  2.0  0.647208         0.659467   
       20     mio_mul_ce   300        27.0  2.0  0.657198         0.666021   
              mio_mul_f1   300        27.0  2.0  0.654267         0.663500   

                                recall_macro  f1-score_macro  load_time  \
sensor inputs filter_lossf gen                                            
25     10     mio_mae      300      0.666588        0.654353   0.810910   
              mio_mul_ap   300      0.749018        0.701168   0.639301   
       20     mio_mul_ce   300      0.749809        0.704884   0.676308   
              mio_mul_f1   300      0.749613        0.703905   0.697805   
26     10     mio_mae      300      0.666588        0.654353   0.745747   
              mio_mul_ap   300      0.749018        0.701168   0.643712   
       20     mio_mul_ce   300      0.749809        0.704884   0.708122   
              mio_mul_f1   300      0.749613        0.703905   0.681223   
27     10     mio_mae      300      0.666588        0.654353   0.755189   
              mio_mul_ap   300      0.749018        0.701168   0.638194   
       20     mio_mul_ce   300      0.749809        0.704884   0.678026   
              mio_mul_f1   300      0.749613        0.703905   0.702822   
28     10     mio_mae      300      0.666588        0.654353   0.772746   
              mio_mul_ap   300      0.749018        0.701168   0.664096   
       20     mio_mul_ce   300      0.749809        0.704884   0.677200   
              mio_mul_f1   300      0.749613        0.703905   0.706796   

                                pred_time  scoring_time  
sensor inputs filter_lossf gen                           
25     10     mio_mae      300   0.286397      0.141541  
              mio_mul_ap   300   0.258706      0.141954  
       20     mio_mul_ce   300   0.181867      0.140733  
              mio_mul_f1   300   0.152568      0.140082  
26     10     mio_mae      300   0.210009      0.140740  
              mio_mul_ap   300   0.198854      0.143496  
       20     mio_mul_ce   300   0.181508      0.140423  
              mio_mul_f1   300   0.154231      0.145713  
27     10     mio_mae      300   0.227262      0.144891  
              mio_mul_ap   300   0.211062      0.144325  
       20     mio_mul_ce   300   0.177813      0.144143  
              mio_mul_f1   300   0.168475      0.148467  
28     10     mio_mae      300   0.246342      0.143031  
              mio_mul_ap   300   0.232856      0.143281  
       20     mio_mul_ce   300   0.212078      0.141695  
              mio_mul_f1   300   0.164738      0.142114

In [13]:
df_scores_c3.groupby(['sensor','inputs','filter_lossf','gen']).mean()

centremost  exp  AP_macro  precision_macro  \
sensor inputs filter_lossf gen                                               
27     10     mio_mae      300        27.0  2.0  0.572291         0.584598   
              mio_mul_ap   300        27.0  2.0  0.644627         0.700889   
       20     mio_mul_ce   300        27.0  2.0  0.657081         0.640546   
              mio_mul_f1   300        27.0  2.0  0.633957         0.701682   

                                recall_macro  f1-score_macro  load_time  \
sensor inputs filter_lossf gen                                            
27     10     mio_mae      300      0.666602        0.622737   0.844805   
              mio_mul_ap   300      0.749143        0.724011   0.670848   
       20     mio_mul_ce   300      0.749794        0.690866   0.725695   
              mio_mul_f1   300      0.734126        0.717071   0.727051   

                                pred_time  scoring_time  
sensor inputs filter_lossf gen                           
27     10     mio_mae      300   0.183250      0.133920  
              mio_mul_ap   300   0.171680      0.133942  
       20     mio_mul_ce   300   0.157275      0.133387  
              mio_mul_f1   300   0.133403      0.134086

#### Condensed results for each cluster for each loss function

In [37]:
df_scores_c1.groupby(['inputs','filter_lossf','gen']).mean().sort_values('f1-score_macro',ascending=False)

centremost  sensor  exp  AP_macro  precision_macro  \
inputs filter_lossf gen                                                       
10     mio_mul_ap   300         4.0    11.5  2.0  0.898443         0.989974   
       mio_mul_ce   300         4.0    11.5  2.0  0.884935         0.980025   
       mio_mae      300         4.0    11.5  2.0  0.827509         0.920668   
       mio_mul_f1   300         4.0    11.5  2.0  0.775767         0.868959   

                         recall_macro  f1-score_macro  load_time  pred_time  \
inputs filter_lossf gen                                                       
10     mio_mul_ap   300      0.899401        0.942516   0.603672   0.099716   
       mio_mul_ce   300      0.895339        0.935709   0.607851   0.125689   
       mio_mae      300      0.897547        0.906746   0.737504   0.081748   
       mio_mul_f1   300      0.895006        0.879631   0.613779   0.158965   

                         scoring_time  
inputs filter_lossf gen                
10     mio_mul_ap   300      0.131764  
       mio_mul_ce   300      0.134481  
       mio_mae      300      0.131925  
       mio_mul_f1   300      0.132613

In [38]:
df_scores_c2.groupby(['inputs','filter_lossf','gen']).mean().sort_values('f1-score_macro',ascending=False)

,,,centremost,sensor,exp,AP_macro,precision_macro,recall_macro,f1-score_macro,load_time,pred_time,scoring_time
inputs,filter_lossf,gen,,,,,,,,,,
50,mio_mul_ap,300,14.0,16.0,2.0,0.777253,0.823138,0.846924,0.834851,0.824530,0.129600,0.104245
30,softmax_mul_f1,300,14.0,16.0,2.0,0.899417,0.824711,0.843085,0.833658,0.720432,0.174462,0.127245
10,softmax_mul_ce,300,14.0,16.0,2.0,0.877634,0.806483,0.812049,0.809210,0.761916,0.082933,0.115739
30,softmax_mae,300,14.0,16.0,2.0,0.639684,0.709186,0.563846,0.618639,0.715762,0.106333,0.124109


In [39]:
df_scores_c3.groupby(['inputs','filter_lossf','gen']).mean().sort_values('f1-score_macro',ascending=False)

centremost  sensor  exp  AP_macro  precision_macro  \
inputs filter_lossf gen                                                       
20     mio_mul_ce   300        27.0    26.5  2.0  0.657198         0.666021   
       mio_mul_f1   300        27.0    26.5  2.0  0.654267         0.663500   
10     mio_mul_ap   300        27.0    26.5  2.0  0.647208         0.659467   
       mio_mae      300        27.0    26.5  2.0  0.573145         0.646749   

                         recall_macro  f1-score_macro  load_time  pred_time  \
inputs filter_lossf gen                                                       
20     mio_mul_ce   300      0.749809        0.704884   0.684914   0.188316   
       mio_mul_f1   300      0.749613        0.703905   0.697161   0.160003   
10     mio_mul_ap   300      0.749018        0.701168   0.646326   0.225370   
       mio_mae      300      0.666588        0.654353   0.771148   0.242502   

                         scoring_time  
inputs filter_lossf gen                
20     mio_mul_ce   300      0.141748  
       mio_mul_f1   300      0.144094  
10     mio_mul_ap   300      0.143264  
       mio_mae      300      0.142551

In [12]:
df_scores_c3.groupby(['inputs','filter_lossf','gen']).mean().sort_values('f1-score_macro',ascending=False)

centremost  sensor  exp  AP_macro  precision_macro  \
inputs filter_lossf gen                                                       
10     mio_mul_ap   300        27.0    27.0  2.0  0.644627         0.700889   
20     mio_mul_f1   300        27.0    27.0  2.0  0.633957         0.701682   
       mio_mul_ce   300        27.0    27.0  2.0  0.657081         0.640546   
10     mio_mae      300        27.0    27.0  2.0  0.572291         0.584598   

                         recall_macro  f1-score_macro  load_time  pred_time  \
inputs filter_lossf gen                                                       
10     mio_mul_ap   300      0.749143        0.724011   0.670848   0.171680   
20     mio_mul_f1   300      0.734126        0.717071   0.727051   0.133403   
       mio_mul_ce   300      0.749794        0.690866   0.725695   0.157275   
10     mio_mae      300      0.666602        0.622737   0.844805   0.183250   

                         scoring_time  
inputs filter_lossf gen                
10     mio_mul_ap   300      0.133942  
20     mio_mul_f1   300      0.134086  
       mio_mul_ce   300      0.133387  
10     mio_mae      300      0.133920